In [1]:
#import json
#import pprint
from pathlib import Path
from functools import cache
from IPython.display import display

from shiny import render, ui


import duckdb
import folium
import pandas as pd
import shiny

In [2]:
HEALTHKIT_DB = "../data/healthkit-sqlite-2023-11-17-fix.db"

In [3]:
CAMINO_SQL = """
    SELECT id, duration, sourceName, creationDate, startDate, endDate, workout_statistics, device 
    FROM workouts 
    WHERE sourceName != 'AllTrails' 
    AND (
        (startDate >= '2023-10-10' AND startDate <= '2023-10-17' AND duration > 100) 
        OR 
        (startDate >= '2023-10-16' AND startDate <= '2023-10-17' AND duration < 100)
    )
    """

In [4]:
WALK_SQL = """
    SELECT * FROM workout_points 
    WHERE workout_id = 'WORKOUT_ID'
    """

In [5]:
CAMINO_STATS_SQL = """
    SELECT * FROM workouts WHERE sourceName != 'AllTrails' AND startDate >= '2023-10-16' AND startDate <= '2023-10-17' 
    AND duration < 100
    """

In [6]:
if Path(HEALTHKIT_DB).exists():
    con = duckdb.connect(HEALTHKIT_DB)
    con.install_extension("sqlite")
    con.load_extension("sqlite")

In [7]:
con.sql("PRAGMA show_tables").to_df() ;

In [8]:
stats_df = con.sql(CAMINO_STATS_SQL).to_df()
# pprint.pprint(stats_df["workout_statistics"].iloc[0]) 
# json.loads(stats_df["workout_statistics"].iloc[0])[0]

In [9]:
workouts_df = con.sql(CAMINO_SQL).to_df()

In [10]:
@cache
def get_walk_data(id):
    """
    Fetch and process walk data for a given ID.
    This function is cached to avoid repeated database queries for the same ID.
    """
    walk_df = con.sql(WALK_SQL.replace("WORKOUT_ID", id)).to_df()
    # Any additional processing can be done here
    return walk_df

In [11]:
walk_ids = workouts_df["id"].values.tolist()[1:]   # Discard evening walk in San Sebastin

In [12]:
def load_walk_data(walk_ids):
    for i, id in enumerate(walk_ids):
        #print(f"#{i+1} - Getting data for walk ID: {id}")
        get_walk_data(id)
    return None

In [13]:
load_walk_data(walk_ids)

In [14]:
def update_map(m, df, colour="blue", line_width=3.5):
    points = df[["latitude", "longitude"]].values.tolist()
    folium.PolyLine(points, color=colour, weight=line_width, opacity=1).add_to(m)
    folium.Marker([df["latitude"].iloc[0], df["longitude"].iloc[0]]).add_to(m)
    return m

# Camino del Norte - Mapping

In [15]:
def create_walk_map(walk_ids, colour, line_width):
    m = folium.Map(location=[43.3183, -1.9812], zoom_start=12, tiles="openstreetmap")
    for id in walk_ids:
        walk_df = get_walk_data(id)  # Use the cached function
        m = update_map(m, walk_df, colour, line_width)
    m.fit_bounds(m.get_bounds())
    return m

In [16]:
## {.sidebar}

ui.input_select("line_color", "Choose Line Color", choices=["red", "blue", "green", "yellow"])
ui.input_slider("line_width", "Select Line Width", min=1, max=10, value=3)

<div class="form-group shiny-input-container">
  <label class="control-label" id="line_color-label" for="line_color">Choose Line Color</label>
  <div>
    <select class="shiny-input-select form-select" id="line_color">      <option value="red" selected="">red</option>
      <option value="blue">blue</option>
      <option value="green">green</option>
      <option value="yellow">yellow</option></select>
  </div>
</div>

<div class="form-group shiny-input-container">
  <label class="control-label" id="line_width-label" for="line_width">Select Line Width</label><input class="js-range-slider" id="line_width" data-skin="shiny" data-min="1" data-max="10" data-from="3" data-step="1" data-grid="false" data-grid-snap="false" data-prettify-separator="," data-prettify-enabled="true" data-keyboard="true" data-data-type="number"/>
</div><script type="application/json" data-html-dependency="">{"name": "ionrangeslider", "version": "2.3.1", "source": {"package": "shiny", "subdir": "www/shared/ionrangeslider/"}, "script": [{"src": "js/ion.rangeSlider.min.js"}], "stylesheet": [], "meta": [], "all_files": false, "head": null}</script>
<script type="application/json" data-html-dependency="">{"name": "preset-shiny-ionrangeslider", "version": "1.8.0.9000", "source": {"package": "shiny", "subdir": "www/shared/ionrangeslider/"}, "script": [], "stylesheet": [{"href": "css/ion.rangeSlider.css", "rel": "stylesheet"}], "meta": [], "all_files": false, "head": null}</script>
<script type="application/json" data-html-dependency="">{"name": "strftime", "version": "0.9.2", "source": {"package": "shiny", "subdir": "www/shared/strftime/"}, "script": [{"src": "strftime-min.js"}], "stylesheet": [], "meta": [], "all_files": false, "head": null}</script>

## Camino map

In [17]:
#@render.plot
def display_map_ui():
    try:
        colour = input.line_color()
        width = input.line_width()
    except:
        colour = "blue"
        width = 3 
    map = create_walk_map(walk_ids, colour, width)
    display(map)

In [18]:
# display_map_ui() if removing decorator to check in notebook